In [1]:
import math
from collections import Counter
import random

In [6]:
# Load and preprocess the dataset
def load_data(file_path):
    """Load the dataset and split into sentences and labels."""
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    sentences = []
    labels = []
    for line in lines:
        parts = line.strip().split('\t')
        if len(parts) == 2:
            labels.append(parts[0])
            sentences.append(parts[1])
    
    return sentences, labels
data = (load_data("SMSSpamCollection"))

#Loaded data, we have an array of each sentence, and then array of all of the labels in a row 
# print(data[1])
# print(data[0])


# Preprocessing: Remove punctuation, URLs, numbers, and convert to lowercase
def preprocess_text(text):
    """Cleans text by removing punctuation, URLs, and numbers."""
    cleaned_text = "".join([char.lower() for char in text if char.isalpha() or char.isspace()])
    return cleaned_text

# Step 2: Calculate Term Frequency (TF)
def compute_tf(doc):
    tf = Counter(doc)
    doc_length = len(doc)
    for word in tf:
        tf[word] /= doc_length  # Normalize by document length
    return tf

# Step 3: Calculate Inverse Document Frequency (IDF)
def compute_idf(corpus):
    idf = {}
    total_docs = len(corpus)
    word_doc_count = Counter()
    
    # Count how many documents contain each word
    for doc in corpus:
        unique_words = set(doc)  # Remove duplicates
        for word in unique_words:
            word_doc_count[word] += 1
    
    # Calculate IDF for each word
    for word, doc_count in word_doc_count.items():
        idf[word] = math.log(total_docs / (1 + doc_count))  # Add 1 to avoid division by 0
    return idf

# Step 4: Compute TF-IDF
def compute_tfidf(corpus):
    tfidf = []
    idf = compute_idf(corpus)  # Compute IDF first
    
    for doc in corpus:
        tf = compute_tf(doc)  # Compute TF for this document
        tfidf_doc = {}
        
        for word, tf_value in tf.items():
            tfidf_doc[word] = tf_value * idf[word]  # Compute TF-IDF
        tfidf.append(tfidf_doc)
    
    return tfidf

# Tokenization and feature extraction (TF-IDF)
def tokenize(text):
    """Tokenizes text and removes stopwords (hardcoded)."""
    stopwords = {"the", "is", "in", "and", "to", "of", "a", "i", "it"}  # Minimal stopword set
    tokens = [word for word in text.split() if word not in stopwords and len(word) > 1]
    return tokens




# cleaned = preprocess_text(data[0][0])
# print(cleaned)
# tokenized = tokenize(cleaned)
# print(tokenized)

class LogisticRegression:
    def __init__(self, learning_rate=0.001, lambda_reg=0.1, num_epochs=1000):
        self.learning_rate = learning_rate
        self.lambda_reg = lambda_reg
        self.num_epochs = num_epochs
        self.weights = []
        self.bias = 0


    def calculate_gradient(self, X, y, pred):
        dw = [0] * len(X[0])
        db = 0

        for i in range(len(y)):
            error = pred[i] - y[i]
            for j in range(len(X[0])):
                dw[j] += error * X[i][j]
            db += error

        for i in range(len(X[0])):
            dw[i] = 1/len(y) * dw[i] + 2 * self.lambda_reg * self.weights[i]
        db = 1/len(y) * db

        return dw, db
    

    def obj_func(self, X, y, pred):
        loss = 0

        for i in range(len(y)):
            epsilon = 1e-15  # Small value to avoid log(0)
            pred[i] = max(epsilon, min(1 - epsilon, pred[i]))  # Clamp the value of pred
            loss += y[i] * math.log(pred[i]) + (1 - y[i]) * math.log(1 - pred[i])

        loss = (-1/len(y)) * loss
        reg = 0
        for weight in self.weights:
            reg += weight ** 2

        return loss + self.lambda_reg * reg


    def SGD_training(self, X, y):
        self.weights = [0] * len(X[0])
        for i in range(len(self.weights)):
            self.weights[i] = random.uniform(-0.1, 0.1)

        for epoch in range(self.num_epochs):
            loss = 0
            for i in range(len(y)):
                pred = 0
                for j in range(len(X[0])):
                    pred += X[i][j] * self.weights[j]
                pred += self.bias
                pred = 1 / (1 + math.exp(-pred))
                dw, db = self.calculate_gradient([X[i]], [y[i]], [pred])
                for j in range(len(X[0])):
                    self.weights[j] -= dw[j] * self.learning_rate
                self.bias -= db * self.learning_rate
                loss += self.obj_func([X[i]], [y[i]], [pred])
            print("Loss: ", loss/len(y))
        return
    

    def MbGD_training(self, X, y, pred):
        return
    

    def Evaluate(self):
        return
    

    def train(self, X, y):
        for epoch in range(self.num_epochs):
            pred = [0] * len(y)
            for i in range(len(y)):
                pred[i] = 1 / (1 + math.exp(-(X[i][0] * self.weight + self.bias)))

            self.SGD(X, y, pred)
        return

    def predict(self, X):
        """Add prediction function to LogisticRegression class"""
        predictions = []
        for i in range(len(X)):
            pred = 0
            for j in range(len(X[0])):
                pred += X[i][j] * self.weights[j]
            pred += self.bias
            pred = 1 / (1 + math.exp(-pred))
            predictions.append(1 if pred >= 0.5 else 0)
        return predictions

def evaluate(y_true, y_pred):
    """Computes accuracy, precision, recall, and F1-score."""
    tp = sum(1 for yt, yp in zip(y_true, y_pred) if yt == 1 and yp == 1)
    tn = sum(1 for yt, yp in zip(y_true, y_pred) if yt == 0 and yp == 0)
    fp = sum(1 for yt, yp in zip(y_true, y_pred) if yt == 0 and yp == 1)
    fn = sum(1 for yt, yp in zip(y_true, y_pred) if yt == 1 and yp == 0)
    
    accuracy = (tp + tn) / len(y_true)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return accuracy, precision, recall, f1



def main():
    """Main function to load data, train the model, and print results."""
    file_path = "SMSSpamCollection"  # Update with actual dataset path
    sentences, labels = load_data(file_path)
    preprocessed_sentences = [preprocess_text(sentence) for sentence in sentences]
    tokenized_sentences = [tokenize(sentence) for sentence in preprocessed_sentences]
    tf_idf = compute_tfidf(tokenized_sentences)
    vocab = set()
    for sentence in tokenized_sentences:
        vocab.update(sentence)
    
    model = LogisticRegression(num_epochs=7)
    # TODO: Add TF-IDF feature extraction
    X = []
    for i in range(len(tokenized_sentences)):
        X.append([tf_idf[i][word] if word in tokenized_sentences[i] else 0 for word in vocab])
    y = [0 if label=="ham" else 1 for label in labels]

    model.SGD_training(X, y)
    
    predictions = model.predict(X)
    accuracy, precision, recall, f1 = evaluate(y, predictions)
    
    print("Final Evaluation:")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")


if __name__ == "__main__":
    main()



KeyboardInterrupt: 

In [ ]:
import math
import random
from collections import Counter

def load_and_split_data(file_path, train_ratio=0.7, val_ratio=0.15):
    """
    Loads text data from a file and splits it into training, validation, and test sets.
    """
    print(f"Reading data from {file_path}...")
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    print(f"Total number of samples loaded: {len(lines)}")
    
    # Set random seed for reproducibility
    random.seed(42)
    random.shuffle(lines)
    
    # Calculate split points
    train_end = int(len(lines) * train_ratio)
    val_end = int(len(lines) * (train_ratio + val_ratio))
    
    # Split data
    train_data = lines[:train_end]
    val_data = lines[train_end:val_end]
    test_data = lines[val_end:]
    
    # Print data distribution
    print("\nData Distribution:")
    print(f"Training samples: {len(train_data)}")
    print(f"Validation samples: {len(val_data)}")
    print(f"Test samples: {len(test_data)}")
    
    def split_data(data):
        sentences = []
        labels = []
        for line in data:
            if '\t' in line:  # Check if line contains tab separator
                label, text = line.strip().split('\t', 1)
                labels.append(label)
                sentences.append(text)
        return sentences, labels
    
    return split_data(train_data) + split_data(val_data) + split_data(test_data)

# [Previous preprocess_text and tokenize functions remain the same]

class LogisticRegression:
    def __init__(self, learning_rate=0.01, lambda_reg=0.1, num_epochs=100, 
                 early_stop_threshold=1e-4, batch_size=32):
        self.learning_rate = learning_rate
        self.lambda_reg = lambda_reg
        self.num_epochs = num_epochs
        self.early_stop_threshold = early_stop_threshold
        self.batch_size = batch_size
        self.weights = None
        self.bias = 0
        self.training_history = []
    
    def train_sgd(self, X, y):
        """Train using Stochastic Gradient Descent with detailed logging."""
        if not self.weights:
            self.weights = [random.uniform(-0.1, 0.1) for _ in range(len(X[0]))]
        
        class_weights = self.compute_class_weights(y)
        n_samples = len(y)
        prev_loss = float('inf')
        
        print("\nTraining with SGD:")
        print("Epoch\tLoss\t\tΔLoss")
        
        for epoch in range(self.num_epochs):
            indices = list(range(n_samples))
            random.shuffle(indices)
            
            total_loss = 0
            for i in indices:
                # Forward pass
                z = sum(X[i][j] * self.weights[j] for j in range(len(self.weights))) + self.bias
                pred = self.sigmoid(z)
                
                # Compute weighted error
                weight = class_weights[y[i]]
                error = weight * (pred - y[i])
                
                # Update weights and bias
                for j in range(len(self.weights)):
                    gradient = error * X[i][j] + 2 * self.lambda_reg * self.weights[j]
                    self.weights[j] -= self.learning_rate * gradient
                self.bias -= self.learning_rate * error
                
                total_loss += self.compute_loss(X, y, [i])
            
            avg_loss = total_loss / n_samples
            loss_change = prev_loss - avg_loss
            self.training_history.append(avg_loss)
            
            if epoch % 5 == 0:  # Print every 5 epochs
                print(f"{epoch}\t{avg_loss:.6f}\t{loss_change:.6f}")
            
            if epoch > 0 and abs(loss_change) < self.early_stop_threshold:
                print(f"\nEarly stopping at epoch {epoch}")
                break
                
            prev_loss = avg_loss

def main():
    # Load and split data
    print("Step 1: Loading and splitting data...")
    train_sentences, train_labels, val_sentences, val_labels, test_sentences, test_labels = \
        load_and_split_data("SMSSpamCollection")
    
    # Print label distribution
    for dataset_name, labels in [("Training", train_labels), 
                               ("Validation", val_labels), 
                               ("Test", test_labels)]:
        spam_count = sum(1 for label in labels if label == "spam")
        ham_count = len(labels) - spam_count
        print(f"\n{dataset_name} set distribution:")
        print(f"Spam: {spam_count} ({spam_count/len(labels)*100:.1f}%)")
        print(f"Ham: {ham_count} ({ham_count/len(labels)*100:.1f}%)")
    
    print("\nStep 2: Preprocessing and tokenizing text...")
    train_processed = [tokenize(preprocess_text(text)) for text in train_sentences]
    val_processed = [tokenize(preprocess_text(text)) for text in val_sentences]
    test_processed = [tokenize(preprocess_text(text)) for text in test_sentences]
    
    print("\nStep 3: Extracting TF-IDF features...")
    vectorizer = TfidfVectorizer()
    vectorizer.fit(train_processed)
    print(f"Vocabulary size: {len(vectorizer.vocabulary)}")
    
    X_train = vectorizer.transform(train_processed)
    X_val = vectorizer.transform(val_processed)
    X_test = vectorizer.transform(test_processed)
    
    # Convert labels
    y_train = [1 if label == "spam" else 0 for label in train_labels]
    y_val = [1 if label == "spam" else 0 for label in val_labels]
    y_test = [1 if label == "spam" else 0 for label in test_labels]
    
    print("\nStep 4: Cross-validation for lambda selection...")
    lambda_values = [0.001, 0.01, 0.1, 1.0]
    best_lambda = cross_validate(X_train, y_train, k_folds=5, lambda_values=lambda_values)
    print(f"Best lambda from cross-validation: {best_lambda}")
    
    print("\nStep 5: Training final model...")
    model = LogisticRegression(lambda_reg=best_lambda, num_epochs=100)
    model.train_sgd(X_train, y_train)
    
    print("\nStep 6: Evaluating on test set...")
    test_pred = model.predict(X_test)
    accuracy, precision, recall, f1 = evaluate(y_test, test_pred)
    
    print("\nFinal Test Results:")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")

if __name__ == "__main__":
    main()

Reading data from SMSSpamCollection...
Total number of samples loaded: 5574


KeyboardInterrupt: 

In [18]:
import math
import random
from collections import Counter

def load_and_split_data(file_path, train_ratio=0.7, val_ratio=0.15):
    """
    Loads text data from a file and splits it into training, validation, and test sets.
    Args:
        file_path (str): Path to the data file
        train_ratio (float): Proportion of data for training
        val_ratio (float): Proportion of data for validation
    Returns:
        tuple: Contains train, validation, and test data and labels
    """
    print(f"Reading data from {file_path}...")
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    print(f"Total number of samples loaded: {len(lines)}")
    
    # Count label distribution
    label_counts = Counter(line.split('\t')[0] for line in lines)
    print("\nLabel Distribution:")
    for label, count in label_counts.items():
        print(f"{label}: {count} ({count/len(lines)*100:.2f}%)")
    
    # Set random seed for reproducibility
    random.seed(42)
    random.shuffle(lines)
    
    # Calculate split points
    train_end = int(len(lines) * train_ratio)
    val_end = int(len(lines) * (train_ratio + val_ratio))
    
    # Split data
    train_data = lines[:train_end]
    val_data = lines[train_end:val_end]
    test_data = lines[val_end:]
    
    def split_data(data):
        sentences = []
        labels = []
        for line in data:
            if '\t' in line:
                label, text = line.strip().split('\t', 1)
                # Convert 'spam' to 1 and 'ham' to 0
                label_int = 1 if label.lower() == 'spam' else 0
                labels.append(label_int)
                sentences.append(text)
        return sentences, labels
    
    return split_data(train_data) + split_data(val_data) + split_data(test_data)

def preprocess_text(text):
    """
    Cleans text by removing special characters and converting to lowercase.
    Args:
        text (str): Input text to clean
    Returns:
        str: Cleaned text
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = ' '.join(word for word in text.split() 
                   if not word.startswith(('http:', 'https:', 'www.')))
    
    # Keep only letters and spaces
    cleaned_text = ""
    for char in text:
        if char.isalpha() or char.isspace():
            cleaned_text += char
    
    return cleaned_text

def tokenize(text):
    """
    Converts text into tokens, removes stopwords, and applies stemming.
    Args:
        text (str): Input text to tokenize
    Returns:
        list: List of processed tokens
    """
    stopwords = {
        "a", "an", "and", "are", "as", "at", "be", "by", "for", "from", 
        "has", "he", "in", "is", "it", "its", "of", "on", "that", "the", 
        "to", "was", "were", "will", "with", "the", "this", "but", "they",
        "have", "had", "what", "when", "where", "who", "which", "why", "how"
    }
    
    def simple_stem(word):
        """Simple word stemming rules"""
        if len(word) < 4:
            return word
        if word.endswith('ing'):
            return word[:-3]
        elif word.endswith('ed'):
            return word[:-2]
        elif word.endswith('s'):
            return word[:-1]
        return word
    
    words = text.split()
    return [simple_stem(word) for word in words 
            if word not in stopwords and len(word) > 1]

class TfidfVectorizer:
    """
    Converts text documents to TF-IDF feature vectors.
    """
    def __init__(self):
        self.vocabulary = set()
        self.idf = {}
        self.vocab_index = {}
    
    def fit(self, documents):
        """
        Builds vocabulary and computes IDF scores from documents
        Args:
            documents (list): List of tokenized documents
        """
        # Build vocabulary
        for doc in documents:
            self.vocabulary.update(doc)
        
        self.vocab_index = {word: idx for idx, word in enumerate(sorted(self.vocabulary))}
        
        # Compute document frequencies
        doc_freq = Counter()
        for doc in documents:
            doc_words = set(doc)
            for word in doc_words:
                doc_freq[word] += 1
        
        # Calculate IDF scores
        num_docs = len(documents)
        self.idf = {word: math.log((num_docs + 1)/(doc_freq[word] + 1)) + 1 
                   for word in self.vocabulary}
    
    def transform(self, documents):
        """
        Transforms documents into TF-IDF feature vectors
        Args:
            documents (list): List of tokenized documents
        Returns:
            list: List of TF-IDF feature vectors
        """
        X = []
        for doc in documents:
            tf = Counter(doc)
            doc_len = len(doc) if len(doc) > 0 else 1
            
            features = [0.0] * len(self.vocabulary)
            for word in set(doc):
                if word in self.vocab_index:
                    idx = self.vocab_index[word]
                    tf_val = tf[word] / doc_len
                    features[idx] = tf_val * self.idf.get(word, 0)
            
            X.append(features)
        return X

class LogisticRegression:
    def __init__(self, learning_rate=0.01, lambda_reg=0.1, num_epochs=100, 
                 early_stop_threshold=1e-4, batch_size=32):
        self.learning_rate = learning_rate
        self.lambda_reg = lambda_reg
        self.num_epochs = num_epochs
        self.early_stop_threshold = early_stop_threshold
        self.batch_size = batch_size
        self.weights = None
        self.bias = 0
        self.training_history = []
    
    def sigmoid(self, z):
        """Compute sigmoid function"""
        # Clip z to prevent overflow
        z = min(max(z, -100), 100)
        return 1 / (1 + math.exp(-z))
    
    def compute_loss(self, X, y, indices):
        """Compute binary cross-entropy loss with L2 regularization"""
        loss = 0
        for i in indices:
            z = sum(X[i][j] * self.weights[j] for j in range(len(self.weights))) + self.bias
            pred = self.sigmoid(z)
            # Add small epsilon to avoid log(0)
            loss -= (y[i] * math.log(pred + 1e-15) + (1 - y[i]) * math.log(1 - pred + 1e-15))
        
        # Add L2 regularization term
        l2_term = self.lambda_reg * sum(w * w for w in self.weights)
        return (loss / len(indices)) + l2_term
    
    def compute_class_weights(self, y):
        """
        Compute class weights to handle class imbalance
        Args:
            y (list): Labels
        Returns:
            dict: Class weights
        """
        counts = Counter(y)
        total = len(y)
        weights = {
            0: total / (2 * counts[0]) if counts[0] > 0 else 1,
            1: total / (2 * counts[1]) if counts[1] > 0 else 1
        }
        print(f"Class weights: {weights}")
        return weights
    
    def train(self, X, y, method='sgd'):
        """
        Train the model using either SGD or mini-batch gradient descent
        Args:
            X (list): Feature vectors
            y (list): Labels
            method (str): 'sgd' or 'minibatch'
        """
        if not self.weights:
            # Initialize weights with small random values
            self.weights = [random.uniform(-0.1, 0.1) for _ in range(len(X[0]))]
        
        # Compute class weights for balanced training
        class_weights = self.compute_class_weights(y)
        
        print(f"\nTraining with {method.upper()}:")
        print("Epoch\tLoss\t\tΔLoss")
        
        n_samples = len(y)
        prev_loss = float('inf')
        no_improvement_count = 0
        
        for epoch in range(self.num_epochs):
            indices = list(range(n_samples))
            random.shuffle(indices)
            
            if method == 'sgd':
                batch_indices = [[i] for i in indices]
            else:
                batch_indices = [indices[i:i + self.batch_size] 
                               for i in range(0, len(indices), self.batch_size)]
            
            epoch_loss = 0
            for batch in batch_indices:
                weight_gradients = [0] * len(self.weights)
                bias_gradient = 0
                
                for i in batch:
                    # Forward pass
                    z = sum(X[i][j] * self.weights[j] for j in range(len(self.weights))) + self.bias
                    pred = self.sigmoid(z)
                    
                    # Apply class weights to error
                    sample_weight = class_weights[y[i]]
                    error = sample_weight * (pred - y[i])
                    
                    # Accumulate gradients
                    for j in range(len(self.weights)):
                        weight_gradients[j] += error * X[i][j]
                    bias_gradient += error
                
                # Apply updates with regularization
                batch_size = len(batch)
                for j in range(len(self.weights)):
                    reg_gradient = 2 * self.lambda_reg * self.weights[j]
                    self.weights[j] -= self.learning_rate * (weight_gradients[j]/batch_size + reg_gradient)
                self.bias -= self.learning_rate * (bias_gradient/batch_size)
                
                epoch_loss += self.compute_loss(X, y, batch)
            
            avg_loss = epoch_loss / len(batch_indices)
            loss_change = prev_loss - avg_loss
            self.training_history.append(avg_loss)
            
            if epoch % 5 == 0:
                print(f"{epoch}\t{avg_loss:.6f}\t{loss_change:.6f}")
            
            # Early stopping with patience
            if abs(loss_change) < self.early_stop_threshold:
                no_improvement_count += 1
                if no_improvement_count >= 3:  # Wait for 3 epochs of no improvement
                    print(f"\nEarly stopping at epoch {epoch}")
                    break
            else:
                no_improvement_count = 0
            
            prev_loss = avg_loss
    
    def predict(self, X):
        """Make predictions on new data"""
        predictions = []
        for x in X:
            z = sum(x[j] * self.weights[j] for j in range(len(self.weights))) + self.bias
            prob = self.sigmoid(z)
            predictions.append(1 if prob >= 0.5 else 0)
        return predictions

def evaluate(y_true, y_pred):
    """
    Compute evaluation metrics
    Returns:
        tuple: (accuracy, precision, recall, f1)
    """
    tp = sum(1 for yt, yp in zip(y_true, y_pred) if yt == 1 and yp == 1)
    tn = sum(1 for yt, yp in zip(y_true, y_pred) if yt == 0 and yp == 0)
    fp = sum(1 for yt, yp in zip(y_true, y_pred) if yt == 0 and yp == 1)
    fn = sum(1 for yt, yp in zip(y_true, y_pred) if yt == 1 and yp == 0)
    
    accuracy = (tp + tn) / len(y_true)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return accuracy, precision, recall, f1

def perform_cross_validation(X, y, lambda_values, k=5):
    """
    Perform k-fold cross-validation to find the best regularization parameter
    Args:
        X (list): Feature vectors
        y (list): Labels
        lambda_values (list): List of lambda values to try
        k (int): Number of folds
    Returns:
        float: Best lambda value
    """
    print("\nPerforming cross-validation...")
    n_samples = len(y)
    fold_size = n_samples // k
    best_lambda = None
    best_score = -float('inf')
    
    for lambda_val in lambda_values:
        print(f"\nTrying lambda = {lambda_val}")
        scores = []
        
        for fold in range(k):
            # Split data into training and validation
            val_start = fold * fold_size
            val_end = val_start + fold_size
            
            X_val = X[val_start:val_end]
            y_val = y[val_start:val_end]
            X_train = X[:val_start] + X[val_end:]
            y_train = y[:val_start] + y[val_end:]
            
            # Train model
            model = LogisticRegression(lambda_reg=lambda_val, num_epochs=50)
            model.train(X_train, y_train, method='minibatch')
            
            # Evaluate
            y_pred = model.predict(X_val)
            accuracy, _, _, f1 = evaluate(y_val, y_pred)
            scores.append(f1)
        
        avg_score = sum(scores) / len(scores)
        print(f"Average F1-score: {avg_score:.4f}")
        
        if avg_score > best_score:
            best_score = avg_score
            best_lambda = lambda_val
    
    print(f"\nBest lambda value: {best_lambda} (F1-score: {best_score:.4f})")
    return best_lambda

def main():
    # Example usage with a sample input file
    # Assuming input file format: label\ttext
    file_path = "SMSSpamCollection"  # Replace with your data file path
    
    # Step 1: Load and split data
    train_texts, train_labels, val_texts, val_labels, test_texts, test_labels = load_and_split_data(file_path)
    
    # Step 2: Preprocess and tokenize texts
    print("\nPreprocessing and tokenizing texts...")
    train_tokens = [tokenize(preprocess_text(text)) for text in train_texts]
    val_tokens = [tokenize(preprocess_text(text)) for text in val_texts]
    test_tokens = [tokenize(preprocess_text(text)) for text in test_texts]
    
    # Step 3: Create TF-IDF features
    vectorizer = TfidfVectorizer()
    vectorizer.fit(train_tokens)
    
    X_train = vectorizer.transform(train_tokens)
    X_val = vectorizer.transform(val_tokens)
    X_test = vectorizer.transform(test_tokens)
    
    # Step 4: Train and evaluate with both SGD and mini-batch
    for method in ['sgd', 'minibatch']:
        print(f"\nStep 4: Training with {method.upper()}...")
        model = LogisticRegression(
            learning_rate=0.1,  # Increased learning rate
            lambda_reg=0.01,    # Reduced regularization
            num_epochs=35,
            early_stop_threshold=1e-4,
            batch_size=32
        )
        model.train(X_train, train_labels, method=method)
        
        # Evaluate on both validation and test sets
        print(f"\nStep 5: Evaluating {method.upper()} model...")
        
        # Validation set evaluation
        val_pred = model.predict(X_val)
        val_accuracy, val_precision, val_recall, val_f1 = evaluate(val_labels, val_pred)
        print(f"\n{method.upper()} Validation Results:")
        print(f"Accuracy:  {val_accuracy:.4f}")
        print(f"Precision: {val_precision:.4f}")
        print(f"Recall:    {val_recall:.4f}")
        print(f"F1-score:  {val_f1:.4f}")
        
        # Test set evaluation
        test_pred = model.predict(X_test)
        test_accuracy, test_precision, test_recall, test_f1 = evaluate(test_labels, test_pred)
        print(f"\n{method.upper()} Test Results:")
        print(f"Accuracy:  {test_accuracy:.4f}")
        print(f"Precision: {test_precision:.4f}")
        print(f"Recall:    {test_recall:.4f}")
        print(f"F1-score:  {test_f1:.4f}")

if __name__ == "__main__":
    main()

Reading data from SMSSpamCollection...
Total number of samples loaded: 5574

Label Distribution:
ham: 4827 (86.60%)
spam: 747 (13.40%)

Preprocessing and tokenizing texts...

Step 4: Training with SGD...
Class weights: {0: 0.5758783584292885, 1: 3.794747081712062}

Training with SGD:
Epoch	Loss		ΔLoss
0	0.565894	inf
5	0.548030	0.004362
10	0.551706	0.003164
15	0.549920	0.003600
20	0.551242	0.002120
25	0.552730	-0.001959
30	0.554134	-0.004919

Step 5: Evaluating SGD model...

SGD Validation Results:
Accuracy:  0.9187
Precision: 0.6524
Recall:    0.9068
F1-score:  0.7589

SGD Test Results:
Accuracy:  0.9283
Precision: 0.6821
Recall:    0.8957
F1-score:  0.7744

Step 4: Training with MINIBATCH...
Class weights: {0: 0.5758783584292885, 1: 3.794747081712062}

Training with MINIBATCH:
Epoch	Loss		ΔLoss
0	0.826326	inf
5	0.604667	0.011946
10	0.589657	0.001560
15	0.588511	-0.005011
20	0.580150	0.010970
25	0.592079	-0.009504
30	0.588796	-0.002602

Step 5: Evaluating MINIBATCH model...

MINIBATCH 